### Arquivo para Balanceamento do Dataset

In [5]:
import os
import shutil
import random
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
import cv2

In [6]:
def listaImg(path_list):
  lista = {}

  for each in path_list:
    name = each.split("/")
    name = name[len(name)-1]
    all_image = [f for f in os.listdir(each) if os.path.isfile(os.path.join(each, f))]
    
    all_images_path_complete = []    
    for each_img in all_image:
        all_images_path_complete.append(each+ "/" + each_img)
    lista[name] = all_images_path_complete
    

    print(f"Quantidade de imagens no path: {name} é ----> {len(lista[name])}")
  return lista

### Aumento de Dados

In [22]:
# ler todas as imagem do folder

def myaugment(lista_img, qtd):
  
  qtd_img = qtd - len(lista_img)
  
  if qtd_img <= 0:
    print("Essa quantidaded de dados já está presente")
    return
  
  for i in range(qtd_img):
    
    total_img = len(lista_img)
    
    img_aleatoria = random.randint(0, total_img-1)
    
    path = lista_img[img_aleatoria]
    
    id_da_imagem = path.split("/")[-1].replace(".png", "").replace(".jpg", "").replace(".jpeg", "")
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    
    altura, largura = image.shape[:2]

    
    tamanho_desejado = (90, 90)

    
    if (largura, altura) != tamanho_desejado:
      image = cv2.resize(image, tamanho_desejado, interpolation=cv2.INTER_AREA)
    
    # plt.imshow(img)
    
    new_path = path.split("/")
    new_path.pop(len(new_path)-1)
    new_path = "/".join(new_path) + "/"
    
    # graus_random = random.randint(5,330)
    
    transform = A.Compose([
      A.RandomCrop(width=90, height=90),
      A.HorizontalFlip(p=0.5),
      A.ShiftScaleRotate(p=0.5),
    ])
    # transform = A.Compose([
    #   A.RandomCrop(width=90, height=90),
    #   A.HorizontalFlip(p=0.5),
    #   A.ShiftScaleRotate(p=0.5),
    #   A.Perspective(p=0.5),
    #   A.Affine(p=0.7, rotate=graus_random, mode=cv2.BORDER_REPLICATE)
    # ])
    
    trf_image = transform(image=image)['image']
    
    trf_image = cv2.cvtColor(trf_image, cv2.COLOR_RGB2BGR)
    
    cv2.imwrite(new_path + f'augmentation_id_{id_da_imagem}_{i}.png', trf_image)
    i = i + 1
    
  print(f"Gerou: {i} imagens, com as {len(lista_img)} deu no total {len(lista_img) + i}")
  

In [23]:
valor_aumentar = 1000
train_or_test = "train"

path_list = [
 f"../../data/base_treinamento/{train_or_test}/ASC-H",
 f"../../data/base_treinamento/{train_or_test}/ASC-US",
 f"../../data/base_treinamento/{train_or_test}/LSIL",
 f"../../data/base_treinamento/{train_or_test}/HSIL",
 f"../../data/base_treinamento/{train_or_test}/Negative for intraepithelial lesion",
 f"../../data/base_treinamento/{train_or_test}/SCC",
 
]




lista_img = listaImg(path_list)

Quantidade de imagens no path: ASC-H é ----> 592
Quantidade de imagens no path: ASC-US é ----> 388
Quantidade de imagens no path: LSIL é ----> 871
Quantidade de imagens no path: HSIL é ----> 1090
Quantidade de imagens no path: Negative for intraepithelial lesion é ----> 4339
Quantidade de imagens no path: SCC é ----> 103


In [24]:
for each_class in path_list:
  class_name = each_class.split("/")[-1]
  print(class_name)
  myaugment(lista_img=lista_img[f"{class_name}"], qtd=valor_aumentar)

ASC-H
Gerou: 408 imagens, com as 592 deu no total 1000
ASC-US
Gerou: 612 imagens, com as 388 deu no total 1000
LSIL
Gerou: 129 imagens, com as 871 deu no total 1000
HSIL
Essa quantidaded de dados já está presente
Negative for intraepithelial lesion
Essa quantidaded de dados já está presente
SCC
Gerou: 897 imagens, com as 103 deu no total 1000


### Balanceamento excluindo Imagens

In [6]:
path_list = [
 "../../data/base_treinamento/train/Negative for intraepithelial lesion",
 "../../data/base_treinamento/train/Positive for intraepithelial lesion",
]

In [49]:
variavel_trade = "Negative for intraepithelial lesion"

In [50]:
path_list = [
 f"../../data/base_treinamento/test/{variavel_trade}",
 
]

In [51]:

# LISTAR TODAS AS IMAGENS DE UM PATH
lista = {}

for each in path_list:
  name = each.split("/")
  name = name[len(name)-1]
  all_image = [f for f in os.listdir(each) if os.path.isfile(os.path.join(each, f))]
  
  all_images_path_complete = []    
  for each_img in all_image:
      all_images_path_complete.append(each+ "/" + each_img)
  lista[name] = all_images_path_complete
  

  print(f"Quantidade de imagens no path: {name} é ----> {len(lista[name])}")


Quantidade de imagens no path: Negative for intraepithelial lesion é ----> 1356


In [ ]:
diff = len(lista["Negative for intraepithelial lesion"]) - len(lista["Positive for intraepithelial lesion"])
diff

In [52]:
diff = len(lista[variavel_trade]) - 125
diff

1231

In [53]:
# EXCLUINDO QUANTIDADE DE IMAGENS DE UM PATCH
# path_excluido = lista["Positive for intraepithelial lesion"]
# path_excluido = lista["Negative for intraepithelial lesion"]
path_excluido = lista[variavel_trade]

random.shuffle(path_excluido)

for i, each in enumerate(path_excluido):
  if i == diff:
    break
  print(each)
  os.remove(each)
  
  

../../data/base_treinamento/test/Negative for intraepithelial lesion/11184.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/7975.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/9902.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/6892.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/6206.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/3718.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/6380.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/1203.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/3629.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/2785.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/6025.png
../../data/base_treinamento/test/Negative for intraepithelial lesion/4764.png
../../data/base_treinamento/test/Negative for intraepithelial l

In [25]:
valor_deletar = 1000
train_or_test = "train"

path_list = [
 f"../../data/base_treinamento/{train_or_test}/ASC-H",
 f"../../data/base_treinamento/{train_or_test}/ASC-US",
 f"../../data/base_treinamento/{train_or_test}/LSIL",
 f"../../data/base_treinamento/{train_or_test}/HSIL",
 f"../../data/base_treinamento/{train_or_test}/Negative for intraepithelial lesion",
 f"../../data/base_treinamento/{train_or_test}/SCC",
 
]

lista = {}

for each in path_list:
  name = each.split("/")
  name = name[len(name)-1]
  all_image = [f for f in os.listdir(each) if os.path.isfile(os.path.join(each, f))]
  
  all_images_path_complete = []    
  for each_img in all_image:
      all_images_path_complete.append(each+ "/" + each_img)
  lista[name] = all_images_path_complete
  

  # print(f"Quantidade de imagens no path: {name} é ----> {len(lista[name])}")

for k,v in lista.items():
  print(k)
  print(v)
  
  diff = len(v) - valor_deletar
  
  print(f"O Arquivo tem {len(v)} imagens e menos o {valor_deletar} = {diff}")
  
  if diff > 0:
    
    path_excluido = lista[k]
    random.shuffle(path_excluido)

    print(f"Fazendo a exclusão de {diff} imagens no {k} path")
    
    for i, each in enumerate(path_excluido):
      if i == diff:
        break
      # print(each)
      os.remove(each)


ASC-H
['../../data/base_treinamento/train/ASC-H/10.png', '../../data/base_treinamento/train/ASC-H/10000.png', '../../data/base_treinamento/train/ASC-H/10002.png', '../../data/base_treinamento/train/ASC-H/10004.png', '../../data/base_treinamento/train/ASC-H/10006.png', '../../data/base_treinamento/train/ASC-H/10009.png', '../../data/base_treinamento/train/ASC-H/10010.png', '../../data/base_treinamento/train/ASC-H/10011.png', '../../data/base_treinamento/train/ASC-H/10012.png', '../../data/base_treinamento/train/ASC-H/10013.png', '../../data/base_treinamento/train/ASC-H/10015.png', '../../data/base_treinamento/train/ASC-H/10017.png', '../../data/base_treinamento/train/ASC-H/10019.png', '../../data/base_treinamento/train/ASC-H/10021.png', '../../data/base_treinamento/train/ASC-H/10025.png', '../../data/base_treinamento/train/ASC-H/10027.png', '../../data/base_treinamento/train/ASC-H/10029.png', '../../data/base_treinamento/train/ASC-H/10031.png', '../../data/base_treinamento/train/ASC-H/1